In [ ]:
from src.classify3D import generate_data, split_data, transform_data, explore_data

# Radial Scale 1

In [ ]:
gen1 = generate_data.GenerateData(radial_scale=1)

In [ ]:
split1 = split_data.SplitData(gen=gen1)

In [ ]:
transform1 = transform_data.TransformData(split=split1)

In [ ]:
exp1 = explore_data.ExploreData(split=split1)

In [ ]:
# exp1.gen('notebook')

In [ ]:
# exp1.basic_stats_train('notebook')

In [ ]:
# exp1.basic_stats_test('notebook')

In [ ]:
# exp1.basic_stats_val('notebook')

In [ ]:
# exp1.class_stats('notebook')

In [ ]:
# exp1.basic_cor_train('notebook')

In [ ]:
# exp1.basic_cor_test('notebook')

In [ ]:
# exp1.basic_cor_val('notebook')

In [ ]:
# exp1.class_cor('notebook')

In [ ]:
# exp1.pplot('notebook')

In [ ]:
# exp1.dis('notebook')

In [ ]:
# exp1.out('notebook')

In [ ]:
# exp1.nov_test('notebook')

In [ ]:
# exp1.nov_val('notebook')

# Radial Scale 2

In [ ]:
gen2 = generate_data.GenerateData(radial_scale=2)

In [ ]:
split2 = split_data.SplitData(gen=gen2)

In [ ]:
transform2 = transform_data.TransformData(split=split2)

In [ ]:
exp2 = explore_data.ExploreData(split=split2)

In [ ]:
# exp2.gen('notebook')

In [ ]:
# exp2.basic_stats_train('notebook')

In [ ]:
# exp2.basic_stats_test('notebook')

In [ ]:
# exp2.basic_stats_val('notebook')

In [ ]:
# exp2.class_stats('notebook')

In [ ]:
# exp2.basic_cor_train('notebook')

In [ ]:
# exp2.basic_cor_test('notebook')

In [ ]:
# exp2.basic_cor_val('notebook')

In [ ]:
# exp2.class_cor('notebook')

In [ ]:
# exp2.pplot('notebook')

In [ ]:
# exp2.dis('notebook')

In [ ]:
# exp2.out('notebook')

In [ ]:
# exp2.nov_test('notebook')

In [ ]:
# exp2.nov_val('notebook')

# Continue